In [2]:
#Scrapping Start

from modules.scrapper_logic1 import  Scrapper1

task = Scrapper1.get_list(1998)
path = 'download/'
year='1998'

Year 1998 has 363 dataset


In [3]:
#scrapper call
import os



Scrapper1.run(year,task,path)

Downloaded: wi_h1_swe_19980110_v01.cdf

Downloaded: wi_h1_swe_19980103_v01.cdf

Downloaded: wi_h1_swe_19980105_v01.cdf

Downloaded: wi_h1_swe_19980107_v01.cdf

Downloaded: wi_h1_swe_19980101_v01.cdf

Downloaded: wi_h1_swe_19980108_v01.cdf

Downloaded: wi_h1_swe_19980106_v01.cdf

Downloaded: wi_h1_swe_19980109_v01.cdf

Downloaded: wi_h1_swe_19980102_v01.cdf

Downloaded: wi_h1_swe_19980104_v01.cdf

Downloaded: wi_h1_swe_19980112_v01.cdf

Downloaded: wi_h1_swe_19980111_v01.cdf

Downloaded: wi_h1_swe_19980113_v01.cdf

Downloaded: wi_h1_swe_19980116_v01.cdf

Downloaded: wi_h1_swe_19980114_v01.cdf

Downloaded: wi_h1_swe_19980119_v01.cdf

Downloaded: wi_h1_swe_19980118_v01.cdf

Downloaded: wi_h1_swe_19980120_v01.cdf

Downloaded: wi_h1_swe_19980117_v01.cdf

Downloaded: wi_h1_swe_19980115_v01.cdf

Downloaded: wi_h1_swe_19980122_v01.cdf

Downloaded: wi_h1_swe_19980121_v01.cdf

Downloaded: wi_h1_swe_19980124_v01.cdf

Downloaded: wi_h1_swe_19980123_v01.cdf

Downloaded: wi_h1_swe_19980125_v01.cdf



In [4]:
if not os.path.exists(f'{path}/{year}'):
    os.makedirs(f'{path}/{year}')

In [7]:
cyear = '1998'
lisdir = os.listdir(f'download//{cyear}')
len(lisdir[:31])

31

In [3]:
#combine

import os
from modules.combine_cdf import *
import cdflib


cyear = '1998'
lisdir = os.listdir(f'download//{cyear}')

for cdfs in range(len(lisdir[:31])):
    
    cdf1,cdf2,cdf3 = cdflib.CDF(f'download//{cyear}/{lisdir[cdfs]}'),cdflib.CDF(f'download//{cyear}/{lisdir[cdfs+1]}'),cdflib.CDF(f'download//{cyear}/{lisdir[cdfs+2]}')
 
    path = f"download//combined//{cyear}/wi_h1_swe_{lisdir[cdfs].split("_")[3]} - {lisdir[cdfs+2].split("_")[3]}.cdf"
    
    # print(len(cdf1['BX']),len(cdf2['BX']),len(cdf3['BX']))
    result_cdf = merge_cdf(cdf1,cdf2,cdf3)
    combined_cdf = make_cdf(path,result_cdf)

    if cdfs+2 ==len(lisdir)-1:
        break
    
   

        

# testing plot

In [ ]:
import cdflib
from modules.calparam import *
import os
from modules.plotter import Plotting
import matplotlib.pyplot as plt
import concurrent.futures


plt.ioff()
def process_file(cdf_file_name):
    plt_name = cdf_file_name.split("_")[-1].split(".")[0]
    plt_ad = 'download//PLOT//1998'
    print(plt_name)
    cdf_file = cdflib.CDF(f'download//combined//1998/{cdf_file_name}')

    time_range= cdflib.cdfepoch.to_datetime(cdf_file.varget('epoch'))
    BX,BY,BZ,b_mag = bmagnitude(cdf_file)
    p_density = thresold(cdf_file.varget('Proton_Np_moment'),1e4)
    vp = Vp_vec(cdf_file)
    temp = Temperature(cdf_file)
    p_beta = plasma_beta(p_density,b_mag,temp)
    tehta_angle,phi_angle = angels(BX,BY,BZ,b_mag)

    plot  = Plotting(plt_name,plt_ad)

    plot.ipplot(time_range,b_mag,BX,BY,BZ,tehta_angle,phi_angle,vp,p_density,temp,p_beta)

lis_dir = os.listdir(r'download\combined\1998')

# Use ThreadPoolExecutor to parallelize processing and plotting tasks
# with concurrent.futures.ThreadPoolExecutor(10) as executor:
#     executor.map(process_file, lis_dir)
# lis_dir
for cdf_file_path in lis_dir:
    process_file(cdf_file_path)

In [1]:
import cdflib
from modules.plotter import Plotting
from modules.calparam import *
cdf_file = cdflib.CDF(r'download\combined\1998\wi_h1_swe_19980101 - 19980103.cdf')
# path=r'E:\SWE\ICME-ML'
time_range= cdflib.cdfepoch.to_datetime(cdf_file.varget('epoch'))
BX,BY,BZ,b_mag = bmagnitude(cdf_file)
p_density = thresold(cdf_file.varget('Proton_Np_moment'),1e4)
vp = Vp_vec(cdf_file)
temp = Temperature(cdf_file)
p_beta = plasma_beta(p_density,b_mag,temp)
tehta_angle,phi_angle = angels(BX,BY,BZ,b_mag)
plot  = Plotting("19980129 - 19980131")
plot.ipplot(time_range,b_mag,BX,BY,BZ,tehta_angle,phi_angle,vp,p_density,temp,p_beta)

# Testing -> new logic

In [8]:
import datetime
import calendar
import os
import cdflib

lis = [i.split("_")[3] for i in os.listdir(f'download\\1998_test')]

def get_all_dates_in_month(year, month):
    if month == 0:
        start_date = datetime.date(year, 1, 1)
        end_date = datetime.date(year, 12, 31)
    else:
        num_days = calendar.monthrange(year, month)[1]
        start_date = datetime.date(year, month, 1)
        end_date = datetime.date(year, month, num_days)
    all_dates = [start_date + datetime.timedelta(days=i) for i in range((end_date - start_date).days + 1)]
    return [date.strftime("%Y%m%d") for date in all_dates]


all_dates = get_all_dates_in_month(1998, 1)
diff_in_lis = set(all_dates) - set(lis)


In [19]:
import cdflib

cdf_file=cdflib.CDF(r'download\combined\1998\wi_h1_swe_19980101 - 19980103.cdf')

time_range= cdflib.cdfepoch.to_datetime(cdf_file.varget('epoch'))
time_range

array(['1998-01-01T00:01:39.090000', '1998-01-01T00:03:15.141000',
       '1998-01-01T00:04:51.192000', ..., '1998-01-03T23:54:36.271000',
       '1998-01-03T23:56:12.323000', '1998-01-03T23:57:48.374000'],
      dtype='datetime64[us]')

In [20]:
import numpy as np

# Define the start and end times for the day
start_time = np.datetime64('1998-01-01T00:00:00')
end_time = np.datetime64('1998-01-01T23:59:59')  # End of the day

# Define the time interval in microseconds (92 seconds)
time_interval = np.timedelta64(92, 's')  # 92 seconds

# Generate timestamps at 92-second intervals for the specified date range
time_range_list = np.arange(start_time, end_time, time_interval)
time_range_list

dimensions = time_range_list.shape

print("Dimensions of time_range_list:", dimensions)

Dimensions of time_range_list: (940,)


In [21]:
cd = cdflib.cdfwrite.CDF('swea_file1.cdf')
z_var1_spec = {
    'Variable': 'time',
    'Data_Type': 1,
    'Num_Elements': 1,
    'Rec_Vary': False ,
    'Dim_Sizes':(940,),
    'Var_Type': 'zVariable',  # Specify zVariable type
    'Sparse': 'no_sparse' # Assuming non-record varying
}

cd.write_var(z_var1_spec,var_attrs=None, var_data=time_range_list)

cd.close()

ValueError: Missing/invalid required spec for creating variable.

In [22]:
cdf_file=cdflib.CDF('swea_file1.cdf')

In [23]:
cdf_file['time']

ValueError: Variable name 'time' not found.

In [15]:
t = cdflib.cdfepoch.to_datetime(cdf_file.varget('time'))


TypeError: Not sure how to handle type <class 'numpy.ndarray'>